In [1]:
import sys
sys.path.append("../classes")

In [10]:
import pickle
from sklearn.svm import SVC

from sklearn.feature_selection import SelectKBest
from sklearn.decomposition import PCA

import numpy as np, pandas as pd
from geno_classifier import *
from geno_utils import parse_database
from itertools import starmap

import GEOparse

### sample types

01	Primary Solid Tumor	TP<br />
02	Recurrent Solid Tumor	TR<br />
03	Primary Blood Derived Cancer - Peripheral Blood	TB<br />
04	Recurrent Blood Derived Cancer - Bone Marrow	TRBM<br />
05	Additional - New Primary	TAP<br />
06	Metastatic	TM<br />
07	Additional Metastatic	TAM<br />
08	Human Tumor Original Cells	THOC<br />
09	Primary Blood Derived Cancer - Bone Marrow	TBM<br />
10	Blood Derived Normal	NB<br />
11	Solid Tissue Normal	NT<br />
12	Buccal Cell Normal	NBC<br />
13	EBV Immortalized Normal	NEBV<br />
14	Bone Marrow Normal	NBM<br />
15	sample type 15	15SH<br />
16	sample type 16	16SH<br />
20	Control Analyte	CELLC<br />
40	Recurrent Blood Derived Cancer - Peripheral Blood	TRB<br />
50	Cell Lines	CELL<br />
60	Primary Xenograft Tissue	XP<br />
61	Cell Line Derived Xenograft Tissue	XCL<br />
99	sample type 99	99SH<br />

#### https://gdc.cancer.gov/resources-tcga-users/tcga-code-tables/sample-type-codes

In [3]:
healthy_suffix = ('11')
unhealthy_suffix = ('01', '06')

In [4]:
breast_cancer_dataset = pd.read_csv("../datasets/AgilentG4502A_07_3", sep="\t")
breast_cancer_dataset[breast_cancer_dataset.columns[:5]].head()

,sample,TCGA-A8-A0A7-01,TCGA-A8-A07G-01,TCGA-A8-A08R-01,TCGA-A8-A06Y-01
0,RNF14,0.496583,1.004500,-0.149417,1.045250
1,DUOXA1,0.578000,1.695500,0.790000,2.308000
2,UBE2Q2,0.279500,0.795167,0.829667,0.479333
3,RNF10,-0.222250,0.632000,-0.017750,1.457750
4,RNF11,1.131750,0.857750,-0.155500,-0.016500


In [5]:
columns = breast_cancer_dataset.columns

healthy = set(filter(lambda column: column.endswith(healthy_suffix), columns))
unhealthy = set(filter(lambda column: column.endswith(unhealthy_suffix), columns))

### number of samples

In [6]:
print(len(healthy), len(unhealthy))

63 534


In [7]:
healthy & unhealthy

set()

In [8]:
labels_ = [(h, 'healthy') for h in healthy] + [(d, 'unhealthy') for d in unhealthy]

In [11]:
X, y = parse_database(breast_cancer_dataset, labels=dict(labels_), n_jobs=-1)

KeyError: 'IDENTIFIER'

### Flux variability analysis